## $2*m*[0.5 - \frac{1}{1 + e^{{ax}^b}}]$

In [2]:
import math
import numpy as np
import pandas as pd
import itertools

In [3]:
class SRE:
    def __init__(self,a,b,m,mc,mc_ml):
        self.a = a
        self.b = b
        self.m = m
        self.mc = mc
        self.mc_ml = mc_ml
        
    def expandgrid(self):
            product = list(itertools.product(list(np.ravel(self.a)),list(np.ravel(self.b)),list(np.ravel(self.m))))
            df = pd.DataFrame(product)
            arr_a = np.array(df[0])
            arr_b = np.array(df[1])
            arr_m = np.array(df[2])
            return arr_a,arr_b,arr_m
        
    def single_sre(self,a,b,m,mc):
        try:
            z = 1 + math.exp((a * mc)**b)
            f = 2* m * (0.5 - (1/z))
            return f
        except OverflowError:
            z = float('inf')
            return z
    
    def dataframe_of_inputs(self):
        sre_input_df = pd.DataFrame({'a':pd.Series(self.a),
                              'b':pd.Series(self.b),
                              'm':pd.Series(self.m)})
        
        
        array_input = np.meshgrid(self.a,self.b,self.m)
        return array_input,sre_input_df
        
    def multiple_SRE(self):
        fout = {}
        new_a,new_b,new_m = self.expandgrid()
        for i in range(len(self.mc)):
            z = np.exp(np.power(np.multiply(new_a,self.mc[i]),new_b))
            f = np.multiply(2,np.multiply(new_m,np.subtract(0.5,np.divide(1,np.add(1,z)))))
            fout.update({i:f})
        multi_sre_df = pd.DataFrame(fout).T
        multi_sre_df.index = ['mc{}'.format(i) for i in range(len(fout.keys()))]
        new_input_df = pd.DataFrame({'new_a':new_a,
                                    'new_b':new_b,
                                    'new_m':new_m})
        return multi_sre_df,fout,new_input_df

if __name__ == '__main__':
    sre = SRE(a = np.linspace( 0.0001, 0.1,50),
              b = np.linspace(0.5,1,50),
              m = np.arange(0.1,0.3,50),
              mc = np.arange(0, 50,1),
              mc_ml = 50)
    exp = sre.expandgrid()
    multiple_sre_output,fout,new_inputdf = sre.multiple_SRE()
    array_input,df_input = sre.dataframe_of_inputs()
    single_sre_output = []
    for i in range(0,501):
        single_sre_output.append(sre.single_sre(a=1.0,b=1.0,m=1.0,mc=i))

In [4]:
import pandas as pd

In [5]:
data_input = pd.read_excel('./CellSage-based analyses (selected XCEL cells) (003) (version 1).xlsx')

In [6]:
data1 = data_input.iloc[1:,15:18].dropna()

In [7]:
data1.columns = data1.loc[1]
data1 = data1.loc[2:]

In [8]:
data1

1,"Time, Weeks",Cycles,"Capacity loss, %"
2,0,0,0
3,0.0347222,2.1875,1.65181
4,0.0713046,4.49219,2.53616
5,0.107887,6.79688,3.24085
6,0.144469,9.10156,3.84786
...,...,...,...
215,7.79018,490.781,19.8576
216,7.82676,493.086,19.8648
217,7.86334,495.391,19.872
218,7.89993,497.695,19.879


In [13]:
if __name__ == '__main__':
    sre = SRE(a = np.linspace( 0.0001, 0.1,10),
              b = np.linspace(0.5,1,10),
              m = np.arange(0.1,0.3,10),
              mc = list(data1['Cycles']),
              mc_ml = list(data1['Capacity loss, %']))
    exp = sre.expandgrid()
    multiple_sre_output,fout,new_inputdf = sre.multiple_SRE()
    array_input,df_input = sre.dataframe_of_inputs()

In [14]:
new_inputdf

,new_a,new_b,new_m
0,0.0001,0.500000,0.1
1,0.0001,0.555556,0.1
2,0.0001,0.611111,0.1
3,0.0001,0.666667,0.1
4,0.0001,0.722222,0.1
...,...,...,...
95,0.1000,0.777778,0.1
96,0.1000,0.833333,0.1
97,0.1000,0.888889,0.1
98,0.1000,0.944444,0.1


In [15]:
multiple_sre_output

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
mc0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mc1,0.000739,0.000463,0.000290,0.000182,0.000114,0.000071,0.000045,0.000028,0.000017,0.000011,...,0.022968,0.021167,0.019499,0.017956,0.016530,0.015213,0.013998,0.012878,0.011845,0.010894
mc2,0.001060,0.000691,0.000450,0.000293,0.000191,0.000125,0.000081,0.000053,0.000034,0.000022,...,0.032311,0.031000,0.029735,0.028515,0.027339,0.026206,0.025117,0.024068,0.023060,0.022091
mc3,0.001303,0.000869,0.000580,0.000387,0.000258,0.000172,0.000115,0.000076,0.000051,0.000034,...,0.039035,0.038291,0.037558,0.036836,0.036125,0.035425,0.034736,0.034057,0.033390,0.032734
mc4,0.001508,0.001022,0.000693,0.000470,0.000318,0.000216,0.000146,0.000099,0.000067,0.000046,...,0.044385,0.044185,0.043985,0.043786,0.043588,0.043391,0.043194,0.042997,0.042802,0.042606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mc213,0.011032,0.009342,0.007908,0.006692,0.005663,0.004791,0.004053,0.003429,0.002900,0.002453,...,0.099819,0.099967,0.099996,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
mc214,0.011057,0.009366,0.007930,0.006713,0.005682,0.004809,0.004069,0.003443,0.002913,0.002465,...,0.099822,0.099967,0.099996,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
mc215,0.011083,0.009390,0.007953,0.006734,0.005701,0.004826,0.004085,0.003457,0.002926,0.002476,...,0.099825,0.099968,0.099996,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
mc216,0.011109,0.009414,0.007975,0.006755,0.005720,0.004843,0.004101,0.003472,0.002939,0.002488,...,0.099827,0.099969,0.099996,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [ ]:
0 vs CL - enet
1 vs CL

# Multicolinearity

In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

In [9]:
sc = StandardScaler()

In [10]:
X_new = sc.fit_transform(multiple_sre_output)

In [11]:
X_new.shape,type(X_new)

((50, 25), numpy.ndarray)

In [12]:
np.array(multiple_sre_output).shape

(50, 25)

In [13]:
multiple_sre_output.shape,type(multiple_sre_output)

((50, 25), pandas.core.frame.DataFrame)

In [14]:
vif = pd.DataFrame()

In [16]:
vif['features'] = multiple_sre_output.keys()
vif['factors']= [variance_inflation_factor(np.array(multiple_sre_output),i) for i in range(25)]
vif['factors1']= [variance_inflation_factor(X_new,i) for i in range(25)]

In [17]:
multiple_sre_output[vif['features'][vif['factors'].isna()]]

,0,5,10,15,20
mc0,0.0,0.0,0.0,0.0,0.0
mc1,0.0,0.0,0.0,0.0,0.0
mc2,0.0,0.0,0.0,0.0,0.0
mc3,0.0,0.0,0.0,0.0,0.0
mc4,0.0,0.0,0.0,0.0,0.0
mc5,0.0,0.0,0.0,0.0,0.0
mc6,0.0,0.0,0.0,0.0,0.0
mc7,0.0,0.0,0.0,0.0,0.0
mc8,0.0,0.0,0.0,0.0,0.0
mc9,0.0,0.0,0.0,0.0,0.0


In [23]:
vif[vif['factors1']<10] # 

,features,factors,factors1
3,3,0.0,1.0


In [24]:
vif[vif['factors']<10]

,features,factors,factors1
1,1,0.0,NaN
2,2,0.0,NaN
3,3,0.0,1.0
4,4,0.0,NaN


In [19]:
vif

,features,factors,factors1
0,0,NaN,NaN
1,1,0.0,NaN
2,2,0.0,NaN
3,3,0.0,1.0
4,4,0.0,NaN
5,5,NaN,NaN
6,6,inf,inf
7,7,inf,inf
8,8,inf,inf
9,9,inf,inf


# PCA

In [25]:
from sklearn.decomposition import PCA

In [26]:
pca = PCA(0.99)

In [27]:
X = pca.fit_transform(multiple_sre_output)

In [28]:
pca.explained_variance_ratio_

array([0.96107989, 0.03864809])

In [29]:
pd.DataFrame(X)

,0,1
0,0.215396,0.048048
1,0.095336,-0.008137
2,0.078462,-0.009981
3,0.067283,-0.010536
4,0.058684,-0.010619
5,0.051597,-0.010471
6,0.045518,-0.010192
7,0.040163,-0.009833
8,0.035358,-0.009422
9,0.030986,-0.008978


In [30]:
new_inputdf

,new_a,new_b,new_m
0,0.0004,0.00,0.00
1,0.0004,0.00,0.15
2,0.0004,0.00,0.30
3,0.0004,0.00,0.45
4,0.0004,0.00,0.60
5,0.0004,0.15,0.00
6,0.0004,0.15,0.15
7,0.0004,0.15,0.30
8,0.0004,0.15,0.45
9,0.0004,0.15,0.60
